1. 문서의 내용을 읽는다
2. 문서를 쪼갠다
    - 토큰수 초과로 답변을 생성하지 못할 수 있고
    - 문서가 길면 (인풋이 길면) 답변 생성이 오래걸림
3. 임베딩 -> 백터 데이터베이스에 저장
4. 질문이 있을 때, 백터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [ ]:
%pip install python-dotenv langchain langchain-openai langchain-community langchain-text-splitters docx2txt

In [15]:
from langchain_community.document_loaders import Docx2txtLoader

loader = Docx2txtLoader('./tax2.docx')
document = loader.load()

In [20]:
#document
len(document)

1

In [ ]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader('./tax2.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [ ]:
#%pip install --upgrade langchain_chroma

In [10]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [11]:
from langchain_chroma import Chroma

#database = Chroma.from_documents(documents=document_list,embedding=embedding)
#database = Chroma.from_documents(documents=document_list,embedding=embedding, collection_name='chroma-tax', persist_directory="./chroma")
database= Chroma(collection_name='chroma-tax', persist_directory="./chroma", embedding_function=embedding)

In [4]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
#query = '제가 제공한 문맥은 어떤 내용인가요? 일부를 제공하여 줄 수 있나요? 혹시 인코딩이 깨지거나 하지는 않았나요? 연봉이 5천만원인 직장인의 소득세를 계산할 수 없나요?'
#retrieved_docs= database.similarity_search(query, k=3)

In [7]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')


In [ ]:
#prompt = f"""[Identity]
#- 당신은 최고의 한국 소득세 전문가 입니다.
#- [Context]를 참고하여 사용자의 질문에 답해주세요.

#[Context]
#{retrieved_docs}

#Question : {query}
#"""

In [ ]:
#ai_message = llm.invoke(prompt)

In [ ]:
#ai_message.content
#%pip install -U langchain langchainhub --quiet

In [ ]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [12]:
#prompt
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [13]:
#ai_message = qa_chain({"query": query})
# 강의에서는 위처럼 진행하지만 업데이트된 LangChain 문법은 `.invoke()` 활용을 권장
ai_message = qa_chain.invoke({"query": query})

In [ ]:
ai_message